In [ ]:
!pip install PyPDF2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import csv
warnings.filterwarnings('ignore')
# NLP Processing
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
!pip install PyPDF2 pandas
from PyPDF2 import PdfReader
from google.colab import files







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Combined Data.csv', on_bad_lines='skip')
df= df.sample(n=8627, random_state=42)

df= df.drop(columns=['Unnamed: 0'])
df.reset_index(drop=True, inplace=True)

df.head()

,statement,status
0,Just as the the title says. I feel like one is...,Depression
1,a blackened sky encroached tugging behind it m...,Depression
2,"It gives you insomnia, which in turn makes you...",Depression
3,"Hello all, I'm a new submitter to this channel...",Normal
4,Thank God the CB is over for Eid,Normal


In [ ]:
mode_status = df['statement'].mode()[0]
df['statement'].fillna(mode_status, inplace=True)
print(df.isnull().sum())
print(mode_status[0])

statement    0
status       0
dtype: int64
w


In [ ]:
# Text preprocessing function
def clean_text(text):
    text = text.lower() # lowercase
    text = "".join([char for char in text if char not in string.punctuation]) # remove punctuation
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')]) # remove stopwords
    lemmatizer = WordNetLemmatizer() # lemmatize the words
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [ ]:
df['clean_statement'] = df['statement'].apply(clean_text)

In [ ]:
# Example: TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_statement'])
y = df['status']

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc , classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import joblib

In [ ]:
# Models to evaluate
model = {'SVM': SVC()}
# Hyperparameter tuning and cross-validation


param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}



if os.path.exists('/content/C:/Users/lenovo/Desktop/Projects/sentiment_svc_model.joblib'):
    # Load the saved model
    loaded_model = joblib.load('/content/C:/Users/lenovo/Desktop/Projects/sentiment_svc_model.joblib')
    # Check if the loaded model is fitted by looking for the 'best_estimator_' attribute
    if hasattr(loaded_model, 'best_estimator_'):
        print("Loaded pre-trained model.")
    else:
        print("Model file exists but does not contain a fitted model. Training...")
        loaded_model.fit(X_train, y_train)  # Fit the loaded model if it's not already fitted
        joblib.dump(loaded_model, '/content/C:/Users/lenovo/Desktop/Projects')  # Save the fitted model
        print("Model trained and saved.")
else:
    # If the model doesn't exist, train and save it
    print("Training model...")
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', refit=True)
    grid_search.fit(X_train, y_train)
    joblib.dump(grid_search, 'sentiment_svc_model.joblib')
    loaded_model = grid_search
    print("Model trained and saved.")


y_pred = loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Training model...
Model trained and saved.
Accuracy: 0.7271147161066048
                      precision    recall  f1-score   support

             Anxiety       0.78      0.83      0.80       121
             Bipolar       0.82      0.71      0.76        89
          Depression       0.64      0.74      0.69       514
              Normal       0.83      0.94      0.88       540
Personality disorder       0.92      0.26      0.40        43
              Stress       0.62      0.29      0.39        83
            Suicidal       0.65      0.51      0.57       336

            accuracy                           0.73      1726
           macro avg       0.75      0.61      0.64      1726
        weighted avg       0.72      0.73      0.71      1726



In [ ]:
from google.colab import files


files.download("sentiment_svc_model.joblib")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install langchain_groq langchain_core langchain_community pypdf  chromadb sentence_transformers

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Shiva? Answer in one line.")
print(result.content)


In [ ]:
import chromadb
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

db_path = "/content/chroma_db"
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  """Creates a vector database from a PDF file."""
  # Create a directory to store the PDF temporarily
  pdf_dir = "/content/drive/MyDrive/pdf_docs"
  loader = DirectoryLoader(pdf_dir, loader_cls=PyPDFLoader) # pass the directory
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


  client = chromadb.PersistentClient(path="/path/to/your/local/directory")
  vector_db = client.from_documents(texts, embeddings)
  vector_db.persist()
  print("ChromaDB created and data saved")
  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain




In [ ]:
def main():
  print("Intializing Chatbot.........")

  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
    # check if vector_db is None
    if vector_db is None:
      return
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)
  query = "" # initialize query to avoid errors in the first loop

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break

        # Clean the user input
    cleaned_input = clean_text(query)

        # Vectorize the new data using the same vectorizer
    X_new = vectorizer.transform([cleaned_input])

        # Make predictions
    new_prediction = loaded_model.predict(X_new)

        # Print the prediction
    print(f" Emotional Status : {new_prediction[0]}")  # Assuming the prediction is a single value

    response = qa_chain.run(query)
    print(f"Chatbot: {response}")
if __name__ == "__main__":
  main()
